In [132]:
import pandas as pd

In [133]:
data = pd.read_csv("Open_Ended_Cars.csv",header = 1)

In [134]:
len(data)

525

In [135]:
data["How old are you? "].median()

29.0

In [136]:
data["What was your household income in the last tax year, before tax was deducted? "].median()

6.0

In [137]:
def median_split(data, i):
    split = len(data)//2
    ordered = data.sort_values(by = i)
    return ordered.iloc[:split], ordered.iloc[split+1:]

In [138]:
lower_age, upper_age = median_split(data, "How old are you? ")
lower_income, upper_income = median_split(data, "What was your household income in the last tax year, before tax was deducted? ")

In [139]:
car_aliases = [['Jeep', 'Fiat', 'Chrysler'],
        ['Subaru', 'Bugeye', 'Scooby'],
        ['Dodge', 'Polara'],
#        ['GMC'],
        ['Tesla'],
#        ['Buick'],
        ['Toyota', 'Yota', 'Camry'],
        ['Honda', 'Accord'],
        ['Nissan', 'Infiniti'],
        ['Chevrolet', 'Chevy'],
        ['Hyundai', 'Tiburon', 'HYU', 'Kia'],
#        ['Ram'],
        ['Mazda', 'Matsuda'],
        ['Renault', 'Dacia'],
        ['Lamborghini', 'Lambo', 'Aventadora', 'Lamborgini'],
        ['Mercedes-Benz', 'Merc', 'Benz', 'Mercedes'],
        ['BMW', 'Beemer', 'Bimmer', 'Beamer'],
        ['Ford', 'Thunderbird', 'Mustang'],
        ['Porsche', 'Porche', 'Porce'],
        ['Audi', '4 Rings'],
        ['Volkswagen', 'VW', 'Volkswagon'],
        ['Ferrari', 'Prancing Horse', 'Scuderia'],
#        ['MG', 'M.G.', 'Morris Garages', 'M.G. Car Company'],
        ['Lexus', 'Lex', 'GX', 'ES 250'],
#        ['Infiniti'],
        ['Volvo']]

In [140]:
car_columns = ["Ford is similar to... ",
"Hyundai is similar to... ",
"BMW is similar to... ",
"Subaru is similar to... ",
"Jeep is similar to... ",
"Toyota is similar to... ",
"Chevrolet is similar to... ",
"Honda is similar to... ",
"audi",
"Mercedes-Benz is similar to... ",
"Tesla is similar to... ",
"Dodge is similar to... ",
"Lexus is similar to... ",
"Porsche is similar to... ",
"Volkswagen is similar to... ",
"Ferrari is similar to... ",
"Nissan is similar to... ",
"Mazda is similar to... ",
"Lamborghini is similar to... ",
"Renault is similar to... ",
"Volvo is similar to... "]

In [141]:
def column_count(data, aliases, prompt):
    alias_pattern = ""
    for i in aliases:
        alias_pattern += i
        alias_pattern += "|"
    alias_pattern = alias_pattern[:-1]   
    series = data[prompt].str.contains(alias_pattern)
    if True in series.unique():
        return series.value_counts()[True]
    return 0

In [142]:
def make_freq_table(data):
    counts = [[] for i in range(len(car_columns))]
    brands = []
    for aliases in car_aliases:
        brands.append(aliases[0])
        for i,prompt in zip(list(range(len(car_columns))), car_columns):
            count = column_count(data, aliases,prompt)
            counts[i].append(count)
    d = dict()
    d["brand"] = brands
    for i,j in zip(car_columns, counts):
        d[i] = j
    df = pd.DataFrame(data=d)
    return df

In [143]:
lower_age_counts = make_freq_table(lower_age)
upper_age_counts = make_freq_table(upper_age)
lower_income_counts = make_freq_table(lower_income)
upper_income_counts = make_freq_table(upper_income)

In [150]:
def most_similar(df):
    return df.set_index("brand").idxmax()

In [159]:
#brand# #low inc# #high# #same?#
def similarity_table(df_low, df_high):
    low = most_similar(df_low)
    high = most_similar(df_high)
    pd_dict = {"Low":low, "High":high}
    df =  pd.DataFrame(pd_dict)
    df["equal"] = df["Low"] == df["High"]
    return df

In [160]:
income_similarity = similarity_table(lower_income_counts, upper_income_counts)
age_similarity = similarity_table(lower_age_counts, upper_age_counts)

In [169]:
def diff_only(sim_table):
    df = sim_table[~sim_table["equal"]]
    return df.drop(columns=["equal"])

In [170]:
income_diff = diff_only(income_similarity)
age_diff = diff_only(age_similarity)

In [171]:
income_diff

,Low,High
Subaru is similar to...,Hyundai,Toyota
Tesla is similar to...,BMW,Mercedes-Benz
Lexus is similar to...,Mercedes-Benz,BMW
Nissan is similar to...,Toyota,Honda
Mazda is similar to...,Honda,Toyota
Renault is similar to...,Volkswagen,Volvo


In [172]:
age_diff

,Low,High
Hyundai is similar to...,Honda,Hyundai
Subaru is similar to...,Toyota,Jeep
Jeep is similar to...,Ford,Dodge
Tesla is similar to...,BMW,Toyota
Lexus is similar to...,Mercedes-Benz,BMW
Volkswagen is similar to...,Volvo,Audi
Nissan is similar to...,Toyota,Honda
Mazda is similar to...,Honda,Nissan
Renault is similar to...,Jeep,Volvo
